# Introduction to operator overloading

Infix operators are the ones that are placed between two operands. 
Unary operator are the operators that receives only a input.
Parenthesis `()`, slicing `[]` and the dot `.` are also operators but will not be discussed here.

Things that cannot be done with operators in python:
- Change the meaning of operators to builtin types
- Create new operators, only overload existing ones
- Overload the following: `is`, `or`, `and`, `not`. Question: they are operators? They seem more like keywords. However, we can overload the binary operators `&`, `|` and `~`

General rule: always return a new object and do not modify the operands.

# Unary operators

The three main unary operators are:
- `-`, implemented by `__neg__`. If `x` is `2`, then `-x == 2`
- `+`, implemented by `__pos__`. In general `x == +x`, but when you want to modify this behavior, use this
- `+`, implemented by `__invert__`. Binary negation of a integer defined as `~x == -(x+1)`. If `x` is `2`, then `~x == -3`.

The `abs()` function could be considered a unary operator as well, and is implemented using `__abs__` dunder method.

We can implement those methods in `Vector` class:

```python
class Vector:
    # A lot of other defs

    def __abs__(self):
        return math.hypot(*self)
    
    def __neg__(self):
        return Vector(-x ofr x in self)

    def __pos__(self):
        Vector(self)
```


# Infix operators

Let's see the implementation of `+` and `*` operators for the Vector class

## Addition operator

We want it to sum the vectors component wise. If a vector is smaller than other, it should be filled with zeros and then summed against the bigger vector
```python
import itertools
class Vector:
    # other definitions

    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b, for a, b in pairs) # every pair is a tuple

v1 = Vector([3, 4, 5])
v1 + (10, 20, 30) # Vector([13.0, 24.0, 35.0])

v2d = Vector2d(1, 2)
v1 + v2d # Vector([4.0, 6.0, 5.0])
```

Because of duck typing, we can even sum a `Vector` and a `Vector2d` or a tuple, because `zip_longest` accepts any iterable. However, with the above version, it only provides this functionality for the right operand. If the order was switched for the examples above, we would receive a error. To fix this we just need to implement `__radd__` dunder method in the `Vector` class. See in the book the reasoning why it exists this function

```python
def __radd__(self, other):
    return self + other # delegates to `__add__`, because the addition is cumolativy in this case.

# When __radd__ just delegates to __add__ we can do
__radd__ = __add__
```

There still some problems with our implementation: if we pass a non-iterable object or a iterable that is composed of elements that cannot be summed to a `float`, we get obscure messages and wrong Error types. To fix this, only change the implementation to:

```python
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b, for a, b in pairs) # every pair is a tuple
        except TypeError:
            return NotImplemented
```

## Scalar multiplication operator

For a multiplication of a `Vector` by a scalar we can implement the following

```python
# inside Vector class
def __mul__(self, scalar):
    try:
        factor = float(scalar)
    except TypeError:
        return NotImplemented
    return Vector(n * scalar for n in self)

# Comulative
def __rmul__(self, scalar):
    return self * scalar

v1 = Vector([1.0, 2.0, 3.0])
v2 = 10.0 * v1 # Vector([10.0, 20.0, 30.0])
```

## Scalar multiplication with operator `@`

Since Python 3.5 (PEP 465), we can use `a @ b` to perform a scalar multiplication (element wise) of two matrices/vectors. 

```python
# inside Vector class

def __matmul__(self, other):
    if (isinstance(other, abc.Sized)) and isinstance(other, abc.Iterable):
        if len(self) == len(other):
            return sum(a * b for a, b in zip(self, other))
        else:
            raise ValueError("@ requires vector of equal length")
    else:
        return NotImplement # Why return and not raise?

def _rmatmul__(self, other):
    return self @ other

def __imatmul__(self, other):
    pass # What's the purpose of this? For something like `x @= y`
```

## Comparison operators

The comparison operators are: `==`, `!=`, `>`, `<`, `>=` and `<=`.

Important:
- For the `==` and `!=`, the reverse operator is called the method itseld or the `id(a) == id(b)`
- The other operators, the reverse method is swapped.

See the table on the book for details.

We can improve the implementation of `Vector`'s `__eq__` knowing about this. The previous implentation was

```python
def __eq__(self, other):
    return (len(self) == len(other) and
        all(a == b for a, b in zip(self, other)))

# which allowed the following
va = Vector([1.0, 2.0, 3.0])
va = Vector(range(1, 4))
va == vb # True, which is okay
vc = Vector([1.0, 2.0])
v2d = Vector2d(1, 2)
vc == v2d # True, not okay, because we can surprise a user with this
t3 = (1, 2, 3)
va == t3 # True, not, also could surprise someone
```

The improved implementation could be

```python
def __eq__(self, other):
    if isinstance(other, Vector):
        return (len(self) == len(other) and
            all(a == b for a, b in zip(self, other)))
    else:
        return NotImplemented

vc = Vector([1.0, 2.0])
v2d = Vector2d(1, 2)
vc == v2d # still True, because we return NotImplemented. Then the `Vector2d.__eq__(v2d, vc) is called, which returns True


Hint: only implement a reverse method if the operand is suppose to work other data types, because in general the reverse method call the direct method.

```


# Augmented attribution operators

Examples of this operators are `+=` and `*=`. Our `Vector` class doesn't implement the `__iadd__()` and `__imul__`, because the internal representation is immutable (?), i. e., a new `Vector` is going to be created anytime we do `v1 += Vector([1.0, 2.0])`, because `+=` is going to be a syntax sugar to `a = a + b`. But if we want to modify the behavior of the operator, we need to do it in a class of mutable objects.

We will do it on a `BingoCage` class. But first, let's see how we want to behave

```python
globe = AddableBingoCage('AEIOU')
globe2 = AddableBingoCage('XYZ')
globe += globe2
len(globe) # 8, because we've added globe2 to globe

globe += ['M', 'N'] # works with any iterable
len(globe) # 10

globe += 1 # TypeError: must be iterable or Tombola type

# For the + operator we want that the right operand to be a AddableCage, but the += is okay
globe3 = globe + ['M', 'N'] # TypeError
```

The implementation would be

```python
# Why we create a subclass of BingoCage instead of modifying it?
class AddableBingoCage(BingoCage):
    def __add__(self, other):
        if isinstace(other, Tombola):
            return AddableBingoCage(self. inspect() + other.self.inspect())
        else:
            return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                raise TypeError("right operand of += must be Tombola or iterable")
        self.load(other_iterable)
        return self
```
